In [104]:
import os
import sys
import numpy as np
from random import randint

class HMM(object): 
    # base class for different HMM models
    def __init__(self, T, O, pi):
        # model is (T, O, pi) where T = Transition probs(hidden_states*hidden_states), 
        # O = Emission Probs(hidden_states*states), pi = initial distribution(hidden_states)               
        if T is None:
            print "Error: You should provide the transition matrix"
            sys.exit() # Read in parameters from the file of model_name
        if O is None:
            print "Error: You should provide the emission matrix"
            sys.exit() # Read in parameters from the file of model_name
        if pi is None:
            print "Error: You should provide the initial probability"
            sys.exit() # Read in parameters from the file of model_name           
        self.pi=pi
        self.T=T
        self.O=O
        self.M=T.shape[1]  # M:number of hidden states of the model
        self.N=O.shape[1]  # N:number of states of the model

        
    def backward(self, obs):
        # This function is for backward algorithm, suppose that T, O, pi 
        #are given, and it calculates a bwk matrix (obs*states).
        # The backward algorithm can be used to calculate the likelihood 
        #of the probability P(Y_{k+1}, ... , Y_n|t_k=C)
        #=sum_q P(Y_{k+2}, ... , Y_n|t_{k+1}=q)P(q|C)P(x_{k+1}|q)
        #The backward probability b is the probability of seeing the observations from 
        #time t + 1 to the end, given that we are in state i at time t
        self.bwk = np.zeros(shape=(len(obs),self.M))
        #[[0 for x in range(self.M)] for y in range(len(obs))] 
        # Initalize bwk to be empty matrix T*M
        # Initialize base cases (t == T)
        self.bwk[len(obs)-1,:]=np.ones(self.M)
        #for y in range(self.M):
            #self.bwk[len(obs)-1][y] = 1 
        for t in reversed(range(len(obs)-1)):
            self.bwk[t,:] = sum((self.bwk[t+1][y1] * self.T[:,y1] * self.O[y1][obs[t+1]]) 
                                    for y1 in range(self.M))                   
        #for t in reversed(range(len(obs)-1)):
            #for y in range(self.M):
                #self.bwk[t][y] = sum((self.bwk[t+1][y1] * self.T[y][y1] * self.O[y1][obs[t+1]]) 
                                    #for y1 in range(self.M))
                #beta_k(C)=\sum_q beta_{k+1}(q)P(q|C)P(w_{k+1}|q)
        prob = sum((self.pi[0][y]* self.O[y][obs[0]] * self.bwk[0][y]) for y in range(self.M))

        return prob 
        #This prob is the likelihood of the input obs   
 

    def forward(self, obs):
        # This function is for forward algorithm, suppose that A, B, pi are given, 
        #and it calculates a fwd matrix (obs*states).
        # The forward algorithm can be used to calculate the likelihood of the model
        #P(Y1, ... , Yn)=sum_t(\prod_i P(Y[i]|t[i])P(t[i]|t[i-1])
        self.fwd = np.zeros(shape=(len(obs),self.M)) 
        #Initalize fwk to be empty matrix, and finally fwd is T*M
        # Initialize base cases (t == 0)
        self.fwd[0,:]=self.pi[0,:] * self.O[:,obs[0]] 
        #for y in range(self.M):
            #self.fwd[0][y] = self.pi[0][y] * self.O[y][obs[0]] 
            #alpha_1(q)=p(w1,t1=q)=P(t1=q|t0)*p(w1|t1=q)
        # Run Forward algorithm for t > 0

        for t in range(1, len(obs)):
            self.fwd[t,:] = sum((self.fwd[t-1][y0] * self.T[y0,:] * self.O[:,obs[t]]) 
                                     for y0 in range(self.M))               
        #for t in range(1, len(obs)):
            #for y in range(self.M):
                #self.fwd[t][y] = sum((self.fwd[t-1][y0] * self.T[y0][y] * self.O[y][obs[t]]) 
                                     #for y0 in range(self.M))
                #alpha_k(q)=\sum_q1 alpha_{k-1}(q1)P(t_k=q|t_{k-1}=q1)P(w_k|t+k=q)
        prob = sum((self.fwd[len(obs) - 1][s]) for s in range(self.M))
        # The likelihood of input equals to the summation of fwd[N][t]
        return prob

    
    def viterbi(self, obs):
    #the task of determining which sequence of variables is the underlying source 
    #of some sequence of observations is called the decoding task
    #Decoding: Given as input an HMM = (A, B, pi) and a sequence of observations 
    #O = Y_1, ... Y_N, find the most probable sequence of states Q = X_1, ... X_T.
    # Goal: find the best path!
    # argmax_t P(Y1, ... Y_N, X_1, ..., X_T|A, B, pi)
        vit = np.zeros(shape=(len(obs),self.M))
        #[[0 for x in range(self.M)] for y in range(len(obs))] 
        # matrix
        path = {} 
        # path
        # Initialize base cases (t == 0)
        vit[0,:] = self.pi[0,:] * self.O[:,obs[0]]
        for y in range(self.M):
            path[y] = [y]           
        #for y in range(self.M):
            #vit[0][y] = self.pi[0][y] * self.O[y][obs[0]]
            #path[y] = [y]
        
        # Run Viterbi for t > 0
        
        for t in range(1, len(obs)):
            newpath = {}
            for y in range(self.M):
                (prob, state) = max((vit[t-1][y0] * self.T[y0][y] * self.O[y][obs[t]], y0) 
                                    for y0 in range(self.M))
                vit[t][y] = prob
                newpath[y] = path[state] + [y]
            # Don't need to remember the old paths
            path = newpath
        n = 0                      
        # if only one element is observed max is sought in the initialization values
        if len(obs)!=1:
            n = t
        (prob, state) = max((vit[n][y], y) for y in range(self.M))
        return (prob, path[state])

    
    
    def forward_backward(self, obs): 
        #Output matrix gamma: gamma[t][y]=P(q_t=j|Y_1, ..., Y_N,A,B,pi)
        #and tensor zi: zi[t][i][j]=P(q_t=i,q_{t+1}=j|Y_1, ..., Y_N,A,B,pi)
        gamma = np.zeros(shape=(len(obs),self.M))
        # this is needed to keep track of finding a state i at a time t for all i and all t
        zi= np.zeros(shape=(len(obs),self.M,self.M))
        #zi = [[[0 for x in range(self.M)] for y in range(self.M)] for z in range(len(obs))]  
        # this is needed to keep track of finding a state i at a time t and j at a time (t+1) 
        #for all i and all j and all t
        # get alpha and beta tables computes
        p_obs = self.forward(obs)
        self.backward(obs)
        # compute gamma values
        for t in range(len(obs)):
            gamma[t,:] = (self.fwd[t,:] * self.bwk[t,:]) / p_obs
            if t == 0:
                self.pi[0][y] = gamma[t][y]
                #gamma[t][y]=P(q_t=j|Y_1, ..., Y_N,A,B,pi)
                #=P(q_t=j,Y_1, ..., Y_N|A,B,pi)/P(Y_1, ..., Y_N|A,B,pi)
                #=alpha_t(j)beta_t(j)/P(Y_1, ..., Y_N|A,B,pi)
                #compute zi values up to T - 1
            if t == len(obs) - 1:
                continue
            for y1 in range(self.M):
                zi[t,:,y1] = self.fwd[t,:] * self.T[y][y1] * self.O[y1][obs[t + 1]] * self.bwk[t + 1][y1] / p_obs
        #zi[t][i][j]=P(q_t=i,q_{t+1}=j|Y_1, ..., Y_N,A,B,pi)       
        #=P(q_t=i,q_{t+1}=j,Y_1, ..., Y_N|A,B,pi)/P(Y_1, ..., Y_N|A,B,pi)
        #=alpha_t(i)a_{ij}b_j(O_{t+1})beta_{t+1}(j)/apha_t(X_T)
        return (gamma,zi)
    
    
    def baum_welch(self,obs):
        # returns model given the initial model and observations  
        #The Baum-Welch algorithm iteratively estimate the counts.
        #We will start with an estimate for the transition and observation probabilities and 
        #then use these estimated probabilities to derive better and better probabilities. 
        #We get our estimated probabilities by computing the forward probability for 
        #an observation and then dividing that probability mass among all the different 
        #paths that contributed to this forward probability.
        gamma = [[0 for x in range(self.M)] for y in range(len(obs))]
        zi = [[[0 for x in range(self.M)] for y in range(self.M)] for z in range(len(obs))]  

        # now that we have gamma and zi let us re-estimate
        (gamma,zi)=self.forward_backward(obs)
        for y in range(self.M):
            for y1 in range(self.M):
                # we will now compute new a_ij
                #a_{ij)=expected number of transitions from state i to state j/expected number 
                #of transitions from state i
                val = sum([zi[t][y][y1] for t in range(len(obs) - 1)]) #
                val /= sum([gamma[t][y] for t in range(len(obs) - 1)])
                self.T[y][y1] = val
        # re estimate gamma
        for y in range(self.M):
            for k in range(self.N): 
                # for all symbols vk
                val = 0.0
                for t in range(len(obs)):
                    if obs[t] == k :
                        val += gamma[t][y]
                val /= sum([gamma[t][y] for t in range(len(obs))])
                self.O[y][k] = val
                    #b_j(v_k)=expected number of times in state j and observing symbol vk/expected 
                    #number of times in state j
        return

In [113]:
M=randint(1,10)
N=randint(1,10)   
T_raw = np.random.random((M, M)) 
row_sums_T= T_raw.sum(axis=1)
T = T_raw / row_sums_T[:, np.newaxis]
# Get transition probability
O_raw = np.random.random((M, N))
row_sums_O = O_raw.sum(axis=1)
O = O_raw / row_sums_O[:, np.newaxis]
# Get emission probability
pi_raw = np.random.random((1, M)) 
row_sums_pi = pi_raw.sum(axis=1)
pi = pi_raw / row_sums_pi[:, np.newaxis]
# Get initial probability
hmm=HMM(T,O,pi)

In [114]:
#M=randint(0,10)
#N=randint(0,10)
#hmm=HMM(M,N)
T=randint(1,10)
observations = []
for i in xrange(0,T):
    observations.append(randint(0,N-1))
#observations=[1,0,1,1]
print "M=", M, "N=", N, "Observations = ", observations

M= 6 N= 7 Observations =  [1, 4, 0, 6, 1, 2, 4, 6, 1]


In [115]:
p1=hmm.backward(observations)
print " Bwk Prob = ", p1

 Bwk Prob =  5.54019329169e-08


In [116]:
p2=hmm.forward(observations)
print " Fwd Prob = ", p2

 Fwd Prob =  5.54019329169e-08


In [117]:
prob, hidden_states = hmm.viterbi(observations)
print "Max Probability = ", prob, " Hidden State Sequence = ", hidden_states

Max Probability =  8.45891913824e-12  Hidden State Sequence =  [4, 3, 0, 3, 4, 3, 0, 3, 4]


In [118]:
(gamma,zi)=hmm.forward_backward(observations)
print "zi= ", zi
print "gamma=", gamma

zi=  [[[ 0.03526616  0.01500547  0.00442107  0.04634348  0.00981426  0.00706527]
  [ 0.02112092  0.00898678  0.00264778  0.02775513  0.00587777  0.00423139]
  [ 0.08544657  0.03635683  0.01071184  0.11228588  0.02377903  0.01711847]
  [ 0.00475606  0.00202366  0.00059623  0.00624996  0.00132357  0.00095283]
  [ 0.15409392  0.06556573  0.01931768  0.20249579  0.04288298  0.03087137]
  [ 0.01311237  0.00557921  0.00164381  0.01723104  0.00364906  0.00262695]]

 [[ 0.0776367   0.02821503  0.00564493  0.11098838  0.02580688  0.01526041]
  [ 0.03409182  0.01238978  0.0024788   0.04873721  0.01133232  0.00670115]
  [ 0.04654357  0.01691504  0.00338416  0.06653806  0.01547135  0.00914869]
  [ 0.08152144  0.02962684  0.00592739  0.11654195  0.02709819  0.016024  ]
  [ 0.04053358  0.01473087  0.00294718  0.05794626  0.0134736   0.00796736]
  [ 0.03446982  0.01252716  0.00250629  0.04927759  0.01145797  0.00677545]]

 [[ 0.05282168  0.08415692  0.00730986  0.07962397  0.04749898  0.03735123]
  [

In [119]:
hmm.baum_welch(observations)
print "The new model parameters after 1 iteration are: "
print "T = ", hmm.T
print "O = ", hmm.O
print "pi = ", hmm.pi

The new model parameters after 1 iteration are: 
T =  [[ 0.21031529  0.22385807  0.03001183  0.27649195  0.15178791  0.10753494]
 [ 0.20735525  0.21055055  0.03239481  0.27169269  0.14645775  0.10500332]
 [ 0.24700807  0.18447582  0.03286735  0.33567606  0.12320514  0.09179195]
 [ 0.20672792  0.21258873  0.02863074  0.26905377  0.14559281  0.102288  ]
 [ 0.22910843  0.19803647  0.03385071  0.30714521  0.13332425  0.09783866]
 [ 0.2199285   0.19631344  0.03166196  0.29736074  0.13498002  0.09933584]]
O =  [[ 0.18082821  0.16639851  0.12736659  0.          0.32040662  0.
   0.20500007]
 [ 0.06270661  0.38998233  0.09171675  0.          0.15132387  0.
   0.30427045]
 [ 0.06859897  0.43203559  0.13847496  0.          0.23731788  0.
   0.1235726 ]
 [ 0.19079145  0.05889552  0.18198251  0.          0.33612202  0.
   0.2322085 ]
 [ 0.07610291  0.53649063  0.04533169  0.          0.12153246  0.
   0.22054231]
 [ 0.06882565  0.34716393  0.12251089  0.          0.18871644  0.
   0.27278309]]
pi 